In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
print(tf.__version__)

In [ ]:
#Reading the train .csv file
data=pd.read_csv('C:\\Users\\SHAHBAZ\\Desktop\\data science\\AppliedAI\\deep learning\\Movies-Poster_Dataset-master\\train.csv')
data.shape

In [ ]:
#taking only 1000rows from the dataset
df_1 = data.iloc[:1000,:]

In [ ]:
df_1.shape

In [ ]:
df_1.head(5)

In [ ]:
#Preprocessing the images in to the same size images
img_width=150
img_heigth=150

x=[]

for i in tqdm(range(1000)):
  path='C:\\Users\\SHAHBAZ\\Desktop\\data science\\AppliedAI\\deep learning\\Movies-Poster_Dataset-master\\Images\\' +data['Id'][i] + '.jpg'
  img= image.load_img(path,target_size=(img_width,img_heigth,3)) #loading the image data into img varaible
  img=image.img_to_array(img)
  #converting the image from 0-255 to 0-1
  img=img/255.0
  x.append(img)
x=np.array(x)

In [ ]:
x.shape

In [ ]:
y=df_1.drop(['Id','Genre'],axis=1)
y=y.to_numpy()
y.shape

In [ ]:
#Train and Test Construction
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.20)

In [ ]:
#data augmentation
data_augmentation = keras.Sequential([ 
   layers.RandomFlip("horizontal"),
   layers.RandomRotation(0.2),
])

In [ ]:
#BUild Model
base_model=keras.applications.Xception(
    weights='imagenet',
    input_shape=(150,150,3),
    include_top=False,
)

#Freeze the base model
base_model.trainable=False

#create new model on top
inputs=keras.Input(shape=(150,150,3))
x=data_augmentation(inputs)
#scale_layer=keras.layers.Rescaling(scale=1/255,offset=0)
#x=scale_layer(x)
x=base_model(x,training=False)
x=keras.layers.GlobalAveragePooling2D()(x)
x=keras.layers.Dropout(0.2)(x)
outputs=keras.layers.Dense(25,activation='sigmoid')(x)
model=keras.Model(inputs,outputs)
model.summary()

In [ ]:
tf.autograph.experimental.do_not_convert
#Train the top layer
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
my_callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
model.fit(x_train,y_train,epochs=40,validation_data=(x_test,y_test),callbacks=my_callbacks)

In [ ]:
#keras fine tuning
base_model.trainable=True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))